In [3]:
from time import time

import torch
import sklearn.datasets
import sklearn.preprocessing
import sklearn.model_selection
import numpy as np
import math
import matplotlib.pyplot as plt

from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST as FMNIST
from torchvision.datasets import EMNIST
import torchvision.transforms as transforms

import Onlinehd

SEED = 1234
device = 'cuda' if torch.cuda.is_available() else 'cpu'
np.random.seed(SEED)
torch.manual_seed(SEED)

In [10]:
import torch.nn as nn

def local_avg(imgs):
    layer = nn.AvgPool2d((3, 3), stride=1, padding = 1)
    outs = imgs.clone().detach().permute((0, 3, 1, 2))
    outs = layer(outs).permute((0, 2, 3, 1))
    return outs

In [4]:
from tensorflow.keras.datasets import fashion_mnist

def load():
    if dataset == 'mnist':
        (x, y), (x_test, y_test) = mnist.load_data()
    elif dataset == 'fashion_mnist':
        (x, y), (x_test, y_test) = fashion_mnist.load_data()
    else:
        temp = EMNIST('./data/EMNIST', split = 'letters', train=True, download=True)
        x = temp.data.unsqueeze(3).numpy().transpose((0,2,1,3))
        y = temp.targets.numpy() - 1
        temp = EMNIST('./data/EMNIST', split='letters', train=False, download=True)
        x_test = temp.data.unsqueeze(3).numpy().transpose((0,2,1,3))
        y_test = temp.targets.numpy() - 1
        
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(y).long()
    x_test = torch.from_numpy(x_test).float()
    y_test = torch.from_numpy(y_test).long().squeeze()

    if len(x.shape) != 3:
        x = x.squeeze(3)
        x_test = x_test.squeeze(3)
    
    return x, x_test, y, y_test

In [7]:
dataset = 'fashion_mnist' # dataset can be 'fashion_mnist', 'mnist', or  'emnist'

x, x_test, y, y_test = load()

In [8]:
x = x.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

scaler = sklearn.preprocessing.Normalizer().fit(x.reshape(-1, 784))
x = torch.tensor(scaler.transform(x.reshape(-1, 784)))
x_test = torch.tensor(scaler.transform(x_test.reshape(-1, 784)))

In [11]:
classes = y.unique().size(0)
features = x.size(1)
model = Onlinehd.OnlineHD(classes, features, dim=10000)

if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x_test = x_test.cuda()
    y_test = y_test.cuda()
    model = model.to('cuda')
    print('Using GPU!')

print('Training...')
t = time()
x_avg = local_avg(x.reshape(-1, 28, 28, 1))
x_avg = x_avg.reshape(-1, 784)
model = model.fit(x_avg, y, bootstrap=1.0, lr=0.035, epochs=100)
t = time() - t

Training...
0
10
20
30
40
50
60
70
80
90


In [12]:
x_avg_test = local_avg(x_test.reshape(-1, 28, 28, 1))
x_avg_test = x_avg_test.reshape(-1, 784)

In [13]:
print('Validating...')
yhat = model(x_avg)
yhat_test = model(x_avg_test)
acc = (y == yhat).float().mean()
acc_test = (y_test == yhat_test).float().mean()
print(f'{acc = :6f}')
print(f'{acc_test = :6f}')
print(f'{t = :6f}')

Validating...
acc = 0.858133
acc_test = 0.843700
t = 91.421526


In [14]:
import pickle

with open('hd_adversarial_sample/FMNIST_HD_JSMA.pickle', 'rb') as f:
    data3 = pickle.load(f)
    JSMA = data3['data']
    LABEL = data3['label']
    
with open('hd_adversarial_sample/FMNIST_HD_DF.pickle', 'rb') as f:
    data2 = pickle.load(f)
    DF = data2['data']
    
import pickle 

with open('hd_adversarial_sample/FMNIST_HD_FGSM.pickle', 'rb') as f:
    data = pickle.load(f)
    FGSM001 = data['data']['0.01']
    FGSM003 = data['data']['0.03']
    FGSM007 = data['data']['0.07']
    FGSM01 = data['data']['0.1']

In [18]:
tmp = FGSM001.reshape(-1, 28, 28, 1)
tmp = local_avg(tmp)
pred = model(tmp.reshape(-1, 784))
print('fgsm_001 : ', sum(pred == y_test)/ len(y_test))

tmp = FGSM003.reshape(-1, 28, 28, 1)
tmp = local_avg(tmp)
pred = model(tmp.reshape(-1, 784))
print('fgsm_003 : ', sum(pred == y_test)/ len(y_test))

tmp = FGSM007.reshape(-1, 28, 28, 1)
tmp = local_avg(tmp)
pred = model(tmp.reshape(-1, 784))
print('fgsm_007 : ', sum(pred == y_test)/ len(y_test))

tmp = FGSM01.reshape(-1, 28, 28, 1)
tmp = local_avg(tmp)
pred = model(tmp.reshape(-1, 784))
print('fgsm_01 : ', sum(pred == y_test)/ len(y_test))



df = torch.from_numpy(DF.reshape(-1, 28, 28, 1))
df = local_avg(df)
pred = model(df.reshape(-1, 784))
print('df : ', sum(pred == LABEL)/ len(LABEL))



jsma = JSMA.reshape(-1, 28, 28, 1)
jsma = local_avg(jsma)
pred = model(jsma.reshape(-1, 784))
print('jsma : ', sum(pred == LABEL)/ len(LABEL))

fgsm_001 :  tensor(0.7205)
fgsm_003 :  tensor(0.4970)
fgsm_007 :  tensor(0.1769)
fgsm_01 :  tensor(0.1273)
df :  tensor(0.7645)
jsma :  tensor(0.8119)
